In [1]:
input_path = '../input/tabular-playground-series-apr-2022/'
output_path = './'

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score

def load_raw_data(train_or_test='train'):
    file_name = f'{input_path}/{train_or_test}.csv'
    df = pd.read_csv(file_name)
    return df

def load_label(train_or_test='train'):
    file_name = input_path + ('train_labels.csv' if train_or_test=='train' else 'sample_submission.csv')
    df = pd.read_csv(file_name)
    return df['state'].values

def competition_metric(y_true, y_score):
    return roc_auc_score(y_true, y_score)

def evaluate(model, X, y):
    return competition_metric(y, model.predict_proba(X)[:, 1])

def submit(arr):
    df = pd.read_csv(f'{input_path}/sample_submission.csv')
    df['state'] = arr
    df.to_csv(f'{output_path}/submission.csv', index=False)

In [3]:
import tensorflow as tf
from tensorflow import keras
from sklearn.base import BaseEstimator, TransformerMixin

class ResNetModel(keras.Model):
    def __init__(self):
        super(ResNetModel, self).__init__()
        self.fns = [
            keras.layers.Conv1D(filters=20, kernel_size=8, padding='same', activation='elu', kernel_regularizer=keras.regularizers.L2(1e-3)), 
            keras.layers.Conv1D(filters=20, kernel_size=8, padding='same', activation='elu', kernel_regularizer=keras.regularizers.L2(1e-3)),
            keras.layers.Conv1D(filters=20, kernel_size=8, padding='same', activation='elu', kernel_regularizer=keras.regularizers.L2(1e-3)),
            keras.layers.AveragePooling1D(2),
            
            keras.layers.Conv1D(filters=20, kernel_size=6, padding='same', activation='elu', kernel_regularizer=keras.regularizers.L2(1e-3)),
            keras.layers.Conv1D(filters=20, kernel_size=6, padding='same', activation='elu', kernel_regularizer=keras.regularizers.L2(1e-3)),
            keras.layers.Conv1D(filters=20, kernel_size=6, padding='same', activation='elu', kernel_regularizer=keras.regularizers.L2(1e-3)),
            keras.layers.AveragePooling1D(2),
            
            keras.layers.Conv1D(filters=20, kernel_size=4, padding='same', activation='elu', kernel_regularizer=keras.regularizers.L2(1e-3)),
            keras.layers.Conv1D(filters=20, kernel_size=4, padding='same', activation='elu', kernel_regularizer=keras.regularizers.L2(1e-3)),
            keras.layers.Conv1D(filters=20, kernel_size=4, padding='same', activation='elu', kernel_regularizer=keras.regularizers.L2(1e-3)),
            keras.layers.AveragePooling1D(3),
            
            keras.layers.GlobalAveragePooling1D(),
            keras.layers.Dense(1, activation='sigmoid')
        ]
        self.bns = [
            keras.layers.BatchNormalization(), 
            keras.layers.BatchNormalization(), 
            
            keras.layers.BatchNormalization(), 
            keras.layers.BatchNormalization(), 
            
            keras.layers.BatchNormalization(), 
            keras.layers.BatchNormalization(), 
            
            keras.layers.BatchNormalization(), 
            keras.layers.BatchNormalization(), 
        ]
        
    def call(self, inputs):
        outputs = inputs
        
        outputs = self.fns[0](outputs)
        res = outputs
        res = self.fns[1](res)
        res = self.fns[2](res)
        outputs += res
        outputs = self.fns[3](outputs)
        
        outputs = self.fns[4](outputs)
        res = outputs
        res = self.fns[5](res)
        res = self.fns[6](res)
        outputs += res
        outputs = self.fns[7](outputs)
        
        outputs = self.fns[8](outputs)
        res = outputs
        res = self.fns[9](res)
        res = self.fns[10](res)
        outputs += res
        outputs = self.fns[11](outputs)
        
        outputs = self.fns[-2](outputs)
        outputs = self.fns[-1](outputs)
        
        return outputs
    
    def predict_proba(self, X):
        return np.concatenate([1-self.predict(X), self.predict(X)], axis=1)

def random_sensor_swap(x, y, random_state=None):
    rng = np.random.default_rng(random_state)
    p_swap = 0.5
    indices = rng.choice(np.arange(x.shape[0]), int(p_swap*x.shape[0]), replace=False)
    x_aug, y_aug = x[indices], y[indices]
    swap_codes = rng.integers(0, 13, (x_aug.shape[0], 2))
    for i in range(x_aug.shape[0]):
        a, b = swap_codes[i]
        x_aug[i, :, [a, b]] = x_aug[i, :, [b, a]]
    x = np.concatenate([x, x_aug], axis=0)
    y = np.concatenate([y, y_aug], axis=0)
    return x, y

def group_splitter(df, nfold=5, random_state=None):
    subject_nums = df['subject'].unique()
    rng = np.random.default_rng(random_state)
    subject_to_setnum = rng.integers(0, nfold, subject_nums.shape[0])
    for i in range(nfold):
        val_subjects = subject_nums[subject_to_setnum == i]
        mask_df_val = df['subject'].isin(val_subjects)
        mask_y_val = mask_df_val.iloc[::60]
        yield mask_df_val, mask_y_val

        
class MyDataAugmenter():
    def fit(self, X, y):
        return self
    
    def transform(self, X, y):
        return X, y

class DF2arr(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        return X.loc[:, 'sensor_00':'sensor_12'].values.reshape(-1, 60, 13)
    
    
class MyPreprocessor(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X = normalize(X)
        return X
    
def normalize(x):
    x = x / (np.linalg.norm(x, axis=1, keepdims=True) + 1e-10)
    return x

In [4]:
import tensorflow as tf
from tensorflow import keras
from sklearn.base import BaseEstimator, TransformerMixin

class RNNThickModel(keras.Model):
    def __init__(self):
        super(RNNThickModel, self).__init__()
        self.fns = [
            keras.layers.LSTM(
                units=256, 
                kernel_regularizer=keras.regularizers.L2(2e-3),
#                 recurrent_regularizer=keras.regularizers.L2(1e-5),
#                 dropout=0.05,
#                 recurrent_dropout=0.01,
                return_sequences=True
            ),
            keras.layers.LSTM(
                units=128,
                kernel_regularizer=keras.regularizers.L2(2e-3),
#                 recurrent_regularizer=keras.regularizers.L2(1e-5),
#                 dropout=0.05,
#                 recurrent_dropout=0.01,
            ),
            keras.layers.Dense(units=32, activation='elu'),
            keras.layers.Dense(units=1, activation='sigmoid')
        ]
    
    def call(self, inputs):
        outputs = inputs
        for layer in self.fns:
            outputs = layer(outputs)
        return outputs
    
    def predict_proba(self, X):
        return np.concatenate([1-self.predict(X), self.predict(X)], axis=1)

    
class MySoftVoter():
    def __init__(self, models, weights=None):
        self.models = models
        if weights is None:
            weights = np.ones((len(models), ))
        weights /= np.sum(weights)
        self.weights = weights
    
    def predict(self, X):
        result = np.zeros((X.shape[0], ), dtype=X.dtype)
        for model, weight in zip(self.models, self.weights):
            add = model.predict(X)
            if len(add.shape) > 1:
                add = add[:, 0]
            result += add * weight
        return result
    
    def predict_proba(self, X):
        return np.stack([1-self.predict(X), self.predict(X)], axis=1)

In [5]:
from sklearn.pipeline import make_pipeline
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import classification_report
cv_scores = []

df = load_raw_data('train')
y = load_label('train')
X = DF2arr().transform(df)
subj_nums = df['subject']

preprocessor = make_pipeline(DF2arr(), MyPreprocessor())

keras.backend.clear_session()
tf.random.set_seed(42)

callbacks = [
    keras.callbacks.EarlyStopping(patience=200, restore_best_weights=True)
]
for mask_df_val, mask_y_val in group_splitter(df, nfold=5, random_state=42):
    df_train, df_val = df[~mask_df_val], df[mask_df_val]
    y_train, y_val = y[~mask_y_val], y[mask_y_val]
    for mask_df_v, mask_y_v in group_splitter(df_train, nfold=5, random_state=42):
        df_t, df_v = df_train[~mask_df_v], df_train[mask_df_v]
        y_t, y_v = y_train[~mask_y_v], y_train[mask_y_v]

    X_t = preprocessor.fit_transform(df_t)
    X_v = preprocessor.transform(df_v)
    X_val = preprocessor.transform(df_val)

    models = [RNNThickModel() for _ in range(5)] + [ResNetModel() for _ in range(5)]
    weights = []
    for model in models:
        with tf.device('gpu:0'):
            model.compile(
                loss='binary_crossentropy', 
                metrics=['AUC'],
                optimizer=keras.optimizers.Adam(1e-3))
            model.fit(
                X_t, y_t, 
                batch_size=1024,
                epochs=500, 
                callbacks=callbacks,
                validation_data=(X_v, y_v),
                verbose=0
            )
            weights.append( model.evaluate(X_val, y_val)[-1] - 0.5 )

    model = MySoftVoter(models, weights)
    print(evaluate(model, X_val, y_val))
    print(classification_report(y_val, (model.predict(X_val) >= 0.5).astype(int), digits=4 ))
    
    cv_scores.append(evaluate(model, X_val, y_val))
print(f'5-fold CV score: {np.mean(cv_scores):.4f}')

2022-09-22 01:26:00.795202: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-22 01:26:00.915799: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-22 01:26:00.916866: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-22 01:26:00.919476: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

161/161 [==============================] - 1s 4ms/step - loss: 0.3087 - auc: 0.9656
0.9745435198452328
              precision    recall  f1-score   support

           0     0.9349    0.9035    0.9190      2592
           1     0.9055    0.9363    0.9207      2559

    accuracy                         0.9198      5151
   macro avg     0.9202    0.9199    0.9198      5151
weighted avg     0.9203    0.9198    0.9198      5151

144/144 [==============================] - 1s 4ms/step - loss: 0.4144 - auc: 0.9394
0.9657345038259397
              precision    recall  f1-score   support

           0     0.9116    0.9104    0.9110      2412
           1     0.9014    0.9026    0.9020      2187

    accuracy                         0.9067      4599
   macro avg     0.9065    0.9065    0.9065      4599
weighted avg     0.9067    0.9067    0.9067      4599

188/188 [==============================] - 1s 4ms/step - loss: 0.3351 - auc: 0.9558
0.969826027266639
              precision    recall  f1-

In [6]:
df_test_final = load_raw_data('test')

X_train = preprocessor.fit_transform(df_train)
X_val = preprocessor.transform(df_val)
X_test_final = preprocessor.transform(df_test_final)

models = [RNNThickModel() for _ in range(5)] + [ResNetModel() for _ in range(5)]
weights = []
for model in models:
    with tf.device('gpu:0'):
        model.compile(
            loss='binary_crossentropy', 
            metrics=['AUC'],
            optimizer=keras.optimizers.Adam(1e-3))
        model.fit(
            X_train, y_train, 
            epochs=500, 
            batch_size=1024,
            callbacks=callbacks,
            validation_data=(X_val, y_val),
            verbose=0
        )
        weights.append( model.evaluate(X_val, y_val)[-1] - 0.5 )
    
model = MySoftVoter(models, weights)
y_pred = model.predict(X_test_final)
submit(y_pred)

155/155 [==============================] - 1s 4ms/step - loss: 0.3091 - auc: 0.9647
